<a href="https://colab.research.google.com/github/Loptt/forest-fire-detector/blob/main/classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fire Dectection Classifier

## Image Preprocessing and Saving

In [2]:
import queue
import logging
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from skimage import io, color
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from PIL import Image
%matplotlib inline

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

train_fire_imgs = []

for i in range(30000): #30000
    try:
        train_fire_imgs.append(io.imread("/content/gdrive/My Drive/SWE/Original_Data/Train/Fire/resized_frame{}.jpg".format(i)))
        print(f'Loaded {i}')
    except:
        # No image found
        pass

Streaming output truncated to the last 5000 lines.
Loaded 13387
Loaded 13388
Loaded 13389
Loaded 13390
Loaded 13391
Loaded 13392
Loaded 13393
Loaded 13394
Loaded 13395
Loaded 13396
Loaded 13397
Loaded 13398
Loaded 13399
Loaded 13400
Loaded 13401
Loaded 13402
Loaded 13403
Loaded 13404
Loaded 13405
Loaded 13406
Loaded 13407
Loaded 13408
Loaded 13409
Loaded 13410
Loaded 13411
Loaded 13412
Loaded 13413
Loaded 13414
Loaded 13415
Loaded 13416
Loaded 13417
Loaded 13418
Loaded 13419
Loaded 13420
Loaded 13421
Loaded 13490
Loaded 13491
Loaded 13492
Loaded 13493
Loaded 13494
Loaded 13495
Loaded 13496
Loaded 13497
Loaded 13498
Loaded 13499
Loaded 13500
Loaded 13501
Loaded 13502
Loaded 13503
Loaded 13504
Loaded 13505
Loaded 13506
Loaded 13507
Loaded 13508
Loaded 13509
Loaded 13510
Loaded 13511
Loaded 13512
Loaded 13513
Loaded 13514
Loaded 13515
Loaded 13516
Loaded 13517
Loaded 13518
Loaded 13519
Loaded 13520
Loaded 13521
Loaded 13522
Loaded 13523
Loaded 13524
Loaded 13525
Loaded 13526
Loaded 13527


In [ ]:
train_no_fire_imgs = []

for i in range(30000): #30000
    try:
        train_no_fire_imgs.append(io.imread("/content/gdrive/My Drive/SWE/Original_Data/Train/No_Fire/lake_resized_lake_frame{}.jpg".format(i)))
    except:
        # No image found
        pass

for i in range(30000): #30000
    try:
        train_no_fire_imgs.append(io.imread("/content/gdrive/My Drive/SWE/Original_Data/Train/No_Fire/resized_frame{}.jpg".format(i)))
    except:
        # No image found
        pass

In [1]:
def calculate_mean_channel(img_lab, n):
    channel = img_lab[:,:,n].flatten()
    return np.mean(channel)

def calculate_rx(img_lab, n, m):
    r1 = img_lab[:,:,n] >= m
    return r1 

def clean_small(mat, threshold):
    visited = np.full(mat.shape, False)
    to_remove = []
    cleaned_mat = np.copy(mat)
    q = queue.Queue()
    size = 0

    for i in range(len(mat)):
        for j in range(len(mat[0])):
            if not mat[i,j]:
                continue
            # For each True pixel perfom BFS and calculate size of cluster
            size = 0
            to_remove = []
            q.put((i,j))
            while not q.empty():
                size += 1
                x, y = q.get()
                if (x < len(mat) and x >= 0 and y < len(mat[0]) and y >= 0) and not visited[x,y] and mat[x,y]:
                    visited[x,y] = True
                    to_remove.append((x,y))
                    q.put((x+1, y))
                    q.put((x-1, y))
                    q.put((x, y+1))
                    q.put((x, y-1))
            # If size is smaller than predefined threshold, remove nodes in to_remove
            # print("Report: Size: {} i: {}, j {}".format(size, i, j))
            if size < threshold:
                for x, y in to_remove:
                    cleaned_mat[x,y] = False
    return cleaned_mat

def apply_mask(img, mask):
    masked = np.copy(img)
    for i in range(len(masked)):
        for j in range(len(masked[0])):
            masked[i,j] = masked[i,j] if mask[i,j] else [0,0,0]
    
    return masked

def enhance_brightness(img, am):
    enhanced = np.copy(img)

    if am < 0:
        return enhanced

    for i in range(len(enhanced)):
        for j in range(len(enhanced[0])):
            for c in range(len(enhanced[0,0])):
                enhanced[i,j,c] = (int(enhanced[i,j,c]) * am) if (int(enhanced[i,j,c]) * am) < 256 else 255
    return enhanced 

def full_transform(img_rgb, enhance, threshold, msg=""):
    img_enc = enhance_brightness(img_rgb, enhance)
    img_lab = color.rgb2lab(img_enc)
    l_m = calculate_mean_channel(img_lab, 0)
    a_m = calculate_mean_channel(img_lab, 1)
    b_m = calculate_mean_channel(img_lab, 2)

    r1 = calculate_rx(img_lab, 0, l_m)
    r2 = calculate_rx(img_lab, 1, a_m)
    r3 = calculate_rx(img_lab, 2, b_m)
    r4 = calculate_rx(img_lab, 2, img_lab[:,:,1])

    r_and = np.logical_and(np.logical_and(np.logical_and(r1, r2), r3), r4)
    r_cleaned = clean_small(r_and, threshold)

    new_img = apply_mask(img_rgb, r_cleaned)

    logging.debug("Transofrmed image " + msg)

    return apply_mask(img_rgb, r_cleaned)

In [ ]:
enhance = 1
threshold = 20

print("Total fire:", len(train_fire_imgs))

train_fire_imgs_transformed = [full_transform(img, enhance, threshold, "Fire {}".format(i)) for i, img in enumerate(train_fire_imgs)]

for i, img in enumerate(train_fire_imgs_transformed):
    img = Image.fromarray(img)
    img.save("data_transformed/Training/Fire/frame_" + str(i) + ".jpg")


oot:Transofrmed image Fire 6097
DEBUG:root:Transofrmed image Fire 6098
DEBUG:root:Transofrmed image Fire 6099
DEBUG:root:Transofrmed image Fire 6100
DEBUG:root:Transofrmed image Fire 6101
DEBUG:root:Transofrmed image Fire 6102
DEBUG:root:Transofrmed image Fire 6103
DEBUG:root:Transofrmed image Fire 6104
DEBUG:root:Transofrmed image Fire 6105
DEBUG:root:Transofrmed image Fire 6106
DEBUG:root:Transofrmed image Fire 6107
DEBUG:root:Transofrmed image Fire 6108
DEBUG:root:Transofrmed image Fire 6109
DEBUG:root:Transofrmed image Fire 6110
DEBUG:root:Transofrmed image Fire 6111
DEBUG:root:Transofrmed image Fire 6112
DEBUG:root:Transofrmed image Fire 6113
DEBUG:root:Transofrmed image Fire 6114
DEBUG:root:Transofrmed image Fire 6115
DEBUG:root:Transofrmed image Fire 6116
DEBUG:root:Transofrmed image Fire 6117
DEBUG:root:Transofrmed image Fire 6118
DEBUG:root:Transofrmed image Fire 6119
DEBUG:root:Transofrmed image Fire 6120
DEBUG:root:Transofrmed image Fire 6121
DEBUG:root:Transofrmed image Fir

In [ ]:
enhance = 1
threshold = 20

print("Total No fire:", len(train_no_fire_imgs))

train_no_fire_imgs_transformed = [full_transform(img, enhance, threshold, "No Fire {}".format(i)) for i, img in enumerate(train_no_fire_imgs)]

for i, img in enumerate(train_no_fire_imgs_transformed):
    img = Image.fromarray(img)
    img.save("data_transformed/Training/No_Fire/frame_" + str(i) + ".jpg")

3891
DEBUG:root:Transofrmed image No Fire 13892
DEBUG:root:Transofrmed image No Fire 13893
DEBUG:root:Transofrmed image No Fire 13894
DEBUG:root:Transofrmed image No Fire 13895
DEBUG:root:Transofrmed image No Fire 13896
DEBUG:root:Transofrmed image No Fire 13897
DEBUG:root:Transofrmed image No Fire 13898
DEBUG:root:Transofrmed image No Fire 13899
DEBUG:root:Transofrmed image No Fire 13900
DEBUG:root:Transofrmed image No Fire 13901
DEBUG:root:Transofrmed image No Fire 13902
DEBUG:root:Transofrmed image No Fire 13903
DEBUG:root:Transofrmed image No Fire 13904
DEBUG:root:Transofrmed image No Fire 13905
DEBUG:root:Transofrmed image No Fire 13906
DEBUG:root:Transofrmed image No Fire 13907
DEBUG:root:Transofrmed image No Fire 13908
DEBUG:root:Transofrmed image No Fire 13909
DEBUG:root:Transofrmed image No Fire 13910
DEBUG:root:Transofrmed image No Fire 13911
DEBUG:root:Transofrmed image No Fire 13912
DEBUG:root:Transofrmed image No Fire 13913
DEBUG:root:Transofrmed image No Fire 13914
DEBUG:

## Loading Preprocessed images
If there are already preprocessed images in data_transformed/ load this to skip the image preprocessing which can take hours.
We are also saving the Fire and No_Fire into a single array with 0 denoting No Fire and 1 denoting Fire.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

data = []

# path = "data_transformed/Training/Fire/" # Local
path = "/content/gdrive/My Drive/Escuela/Octavo Semestre/Fire_Transformed/Training/" # Colab

In [ ]:
for i in range(30000):
    try:
        data.append((np.uint8(io.imread(path + "Fire/frame_{}.jpg".format(i))), 1))
        logging.debug(f"Loading Fire Transformed {i}")
    except:
        # No image found
        pass

In [ ]:
for i in range(30000):
    try:
        data.append((np.uint8(io.imread(path + "No_Fire/frame_{}.jpg".format(i))), 0))
        logging.debug(f"Loading No Fire Transformed {i}")
    except:
        # No image found
        pass

In [ ]:
random.shuffle(data)

train_x = np.array([point[0] for point in data])
train_y = np.array([point[1] for point in data])

print(train_x.shape)
print(train_y.shape)
print(type(train_x))

(20967, 254, 254, 3)
(20967,)


In [ ]:
clf = SVC(kernel="rbf", C=1.0)
clf.fit(train_x, train_y)




MemoryError: Unable to allocate 30.2 GiB for an array with shape (20967, 254, 254, 3) and data type float64